# 1.0 Um problema de classificação de ponta-a-ponta usando NLP

## 1.1 Descrição do dataset

O dataset contém informações gerais de 5.000 processos julgados
nos Juizados Especiais Federais dentro das Seções Judiciárias do Tribunal Regional Federal da 5a Região (https://www.trf5.jus.br). Os dados são oriundos da raspagem da consulta pública processual. Além disso, ele possui 46 colunas, das quais duas possuem texto livre:
"conteudo_sentenca" e "conteudo_acordao".

O dataset pode ser baixado no link a seguir: https://jacob.al/dataset_juizados 

Ao longo dos notebooks, vão ser realizados os seguintes passos:

1. Importação do dataset **(concluído)**
2. Análise exploratória dos dados **(concluído)**
3. Pré-processamento **(concluído)**
4. Verificação dos dados **(concluído)**
5. Segregação dos dados **(concluído)**
6. Treinamento **(concluído)**
7. Teste

<center><img width="600" src="https://drive.google.com/uc?export=view&id=1fKGuR5U5ECf7On6Zo1UWzAIWZrMmZnGc"></center>


## 1.2 Instalação e importação das biblitoecas

In [ ]:
!pip install wandb

In [ ]:
!pip install bertopic

In [24]:
import logging
import pandas as pd
import wandb
import joblib
import re
from sklearn.pipeline import Pipeline, FeatureUnion
from bertopic import BERTopic
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.metrics import fbeta_score, precision_score, recall_score, accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# Login to Weights & Biases
!wandb login --relogin

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


## 1.3 Avaliação do teste

### 1.3.1 Definição das classes base

In [ ]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    # construtor da classe
    def __init__(self, feature_names):
        self.feature_names = feature_names
        
    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        return X[self.feature_names]

def doc_to_list(text_df, list_words):
  for i in range(text_df.shape[0]):
    list_words.append(text_df.loc[i, 'conteudo_sentenca'])

  return list_words

# Tratando variáveis categóricas
class CategoricalTransformer(BaseEstimator, TransformerMixin):

    def __init__(self, new_features=True, colnames=None):
        self.new_features = new_features
        self.colnames = colnames

    def fit(self, X, y=None):
        return self

    def get_feature_names_out(self):
        return self.colnames.tolist()

    def transform(self, X, y=None):
        df = pd.DataFrame(X, columns=self.colnames)

        # Removendo a pontuação
        df['conteudo_sentenca'] = df['conteudo_sentenca'].map(lambda x: re.sub('-', ' ', x))
        df['conteudo_sentenca'] = df['conteudo_sentenca'].map(lambda x: re.sub('[!"#$%&()*+,-./:;<=>?@[\]^_`{|}~]', '', x))

        # Colocando todos os textos em minúsuculo
        df['conteudo_sentenca'] = df['conteudo_sentenca'].map(lambda x: x.lower())

        # Removendo caracteres especiais e stopwords
        stop = stopwords.words('portuguese')

        # palavras comuns em processos jurídicos
        stop.extend(['artigo', 'lei', 'sentença', 'município', 'nacional', '1o', 'art', 
             'autora', 'parte', 'honorários', 'advocatícios', 'termo', 'tempo',
             'justiça', 'etc', 'n°', '°', 'termos', 'parágrafo'])
        pat = r'\b(?:{}/-;)\b'.format('|'.join(stop))
        df['conteudo_sentenca'] = df['conteudo_sentenca'].str.replace(pat, '')

        # Removendo dígitos
        df['conteudo_sentenca'] = df['conteudo_sentenca'].str.replace(r'\d+', ' ')

        # Removendo sub-string
        df['conteudo_sentenca'] = df['conteudo_sentenca'].str.replace(r'\s+', ' ')

        # Pegando os tokens das palavras
        df['conteudo_sentenca'].apply(word_tokenize)
        
        ''' Usada na parte da modelagem com redes neurais
        # Pegando os tokens das palavras
        list_words = []
        list_words = doc_to_list(df, list_words)

        # criando o tokenizer
        tokenizer = create_tokenizer(list_words)

        df = tokenizer.texts_to_matrix(df.loc[:,'conteudo_sentenca'], mode='freq')
        '''
        return df

### 1.3.2 Evaluation

In [ ]:
# global variables

# nome do artefato relacionado ao arquivo de teste
artifact_test_name = "nlp_bolsa/test.csv:latest"

# nome do artefato do modelo
artifact_model_name = "nlp_bolsa/model_export:latest"

In [ ]:
# configuração do logging
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s %(message)s",
                    datefmt='%d-%m-%Y %H:%M:%S')

# objeto
logger = logging.getLogger()

In [ ]:
# inicialização do projeto do wandb
run = wandb.init(project="nlp_bolsa",job_type="test")

wandb: Currently logged in as: morsinaldo. Use `wandb login --relogin` to force relogin


In [ ]:
logger.info("Downloading and reading test artifact")
test_data_path = run.use_artifact(artifact_test_name).file()
df_test = pd.read_csv(test_data_path)

# extraindo a variável alvo
logger.info("Extracting target from dataframe")
x_test = df_test.copy()
y_test = x_test.pop("assunto_cnj")

17-07-2022 21:07:20 Downloading and reading test artifact
17-07-2022 21:07:21 Extracting target from dataframe


In [ ]:
x_test.head()

,conteudo_sentenca
0,SENTENÇA - TIPO C Por força do disposto no art...
1,SENTENÇA Dispensado o relatório nos termos do ...
2,"SENTENÇA I – RELATÓRIO Dispensado, a teor do a..."
3,SENTENÇA 1. RELATÓRIO. Dispensado o relatório ...
4,"SENTENÇA - Tipo C Vistos, etc. I. RELATÓRIO Cu..."


In [21]:
# Download inference artifact
logger.info("Downloading and load the exported model")
model_export_path = run.use_artifact(artifact_model_name).file()
topic_model = joblib.load(model_export_path)

17-07-2022 23:49:23 Downloading and load the exported model


In [25]:
# Pegando as featues categóricas
categorical_features = x_test.select_dtypes("object").columns.to_list()

# Definição dos passos do pipeline categórico
categorical_pipeline = Pipeline(steps=[('cat_selector', FeatureSelector(categorical_features)),
                                       ('cat_transformer', CategoricalTransformer(colnames=categorical_features))
                                       ]
                                )

full_pipeline_preprocessing = FeatureUnion(transformer_list=[('cat_pipeline', categorical_pipeline)
                                                             ]
                              )

new_data_test = full_pipeline_preprocessing.fit_transform(x_test)
df_new_test = pd.DataFrame(new_data_test, columns=['conteudo_sentenca'])
df_new_test.shape

(1500, 1)

In [26]:
# predict
logger.info("Infering")
topics_test, probs_test = topic_model.transform(df_new_test['conteudo_sentenca'])

17-07-2022 23:51:46 Infering


Batches:   0%|          | 0/47 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/numba/np/ufunc/parallel.py:363: NumbaWarning: The TBB threading layer requires TBB version 2019.5 or later i.e., TBB_INTERFACE_VERSION >= 11005. Found TBB_INTERFACE_VERSION = 9107. The TBB threading layer is disabled.
  warnings.warn(problem)


In [27]:
topic_model.visualize_distribution(probs_test[150], min_probability=0.005)

In [28]:
run.finish()